In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

df = pd.read_excel('balanced_symptom_data.xlsx')
print(df['label'].value_counts())

X = df.drop('label', axis=1).values
y = df['label'].values

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

def get_user_input():
    print("Please enter the following details about your condition:")
    try:
        days_of_fever = int(input("How many days have you had a fever (0 if no fever)? "))
        temperature = float(input("What is your temperature in Fahrenheit (e.g., 98.6)? "))
        previous_issues = int(input("Do you have any previous health issues (1 = Yes, 0 = No)? "))
        vomiting = int(input("Are you experiencing vomiting (1 = Yes, 0 = No)? "))
        cold = int(input("Do you have a cold (1 = Yes, 0 = No)? "))
        fatigue = int(input("Are you feeling fatigued or unusually tired (1 = Yes, 0 = No)? "))
        cough = int(input("Do you have a cough (1 = Yes, 0 = No)? "))
        body_pain = int(input("Do you have body pain (1 = Yes, 0 = No)? "))
        user_data = [days_of_fever, temperature, previous_issues, vomiting, cold, fatigue, cough, body_pain]
        return user_data
    except ValueError:
        print("Invalid input. Please enter the correct values.")
        return get_user_input()

def predict_new_case(model, user_data):
    user_data_scaled = scaler.transform(np.array(user_data).reshape(1, -1))
    prediction = model.predict(user_data_scaled)
    
    if prediction >= 2: 
        print("\n!!! Serious condition detected! !!!")
        print("It is advisable to consult a doctor immediately.")
    elif 1 <= prediction < 2: 
        print("\n!!! It seems like a mild case. !!!")
        print("Consider home remedies.")
        print("- Get plenty of rest and avoid strenuous activity.")
        print("- Maintain good hygiene by washing your hands regularly.")
    else:  
        print("\n!!! Normal condition detected. !!!")
        print("No action needed. If symptoms persist or worsen, please consult a doctor.")

def main():
    print("Welcome to the Symptom Checker!")
    user_data = get_user_input()
    predict_new_case(model, user_data)

if __name__ == "__main__":
    main()


label
0    333
1    333
2    333
Name: count, dtype: int64
Mean Squared Error: 0.05
R-squared: 0.93
Welcome to the Symptom Checker!
Please enter the following details about your condition:

!!! Normal condition detected. !!!
No action needed. If symptoms persist or worsen, please consult a doctor.


In [ ]:
import json
import re
import random
import string
import time
import numpy as np
import firebase_admin
from firebase_admin import credentials, firestore
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import joblib

if not firebase_admin._apps:
    cred = credentials.Certificate('medi-ai-bot-firebase-adminsdk-tez3h-83a58427d1.json')
    firebase_admin.initialize_app(cred)
else:
    print("Firebase app already initialized.")

db = firestore.client()

def generate_random_sender():
    return ''.join(random.choices(string.ascii_letters + string.digits, k=10))
def send_question_to_firestore(question):
    sender = generate_random_sender()
    doc_ref = db.collection('botmessages').document()
    timestamp = int(time.time() * 1000)
    doc_ref.set({
        'message': question,
        'sender': sender,
        'timestamp': timestamp
    })
    print(f"Sent question to Firebase: {question}, Sender: {sender}")
    return timestamp

def wait_for_answer_in_firestore(sent_timestamp):
    print(f"Waiting for an answer with a timestamp greater than {sent_timestamp}")
    while True:
        try:
            answers_ref = db.collection('usermessages')\
                .where('timestamp', '>', sent_timestamp)\
                .order_by('timestamp', direction=firestore.Query.ASCENDING)\
                .limit(1)
            answers = answers_ref.stream()
            for answer in answers:
                answer_data = answer.to_dict()
                print(f"Answer received: {answer_data.get('message')}")
                return answer_data.get('message', None)
        except Exception as e:
            print(f"Firestore query failed: {e}. Retrying...")
        time.sleep(2)

def collect_symptom_data_via_firestore():
    questions = [
        "How many days have you had a fever (0 if no fever)?",
        "What is your temperature in Fahrenheit (e.g., 98.6)?",
        "Do you have any previous health issues (1 = Yes, 0 = No)?",
        "Are you experiencing vomiting (1 = Yes, 0 = No)?",
        "Do you have a cold (1 = Yes, 0 = No)?",
        "Are you feeling fatigued or unusually tired (1 = Yes, 0 = No)?",
        "Do you have a cough (1 = Yes, 0 = No)?",
        "Do you have body pain (1 = Yes, 0 = No)?"
    ]

    user_data = []
    for question in questions:
        sent_timestamp = send_question_to_firestore(question)
        answer = wait_for_answer_in_firestore(sent_timestamp)
        if answer is not None:
            try:
                if "temperature" in question:
                    user_data.append(float(answer))
                else:
                    user_data.append(int(answer))
                print(f"Received answer: {answer}")
            except ValueError:
                print(f"Invalid answer received for {question}. Retrying...")
        else:
            print(f"No answer received for {question}. Retrying...")

    return user_data

def predict_new_case(model, scaler, user_data):
    user_data_scaled = scaler.transform(np.array(user_data).reshape(1, -1))
    prediction = model.predict(user_data_scaled)
    
    if prediction >= 2:
        return "\n!!! Serious condition detected! !!!\nIt is advisable to consult a doctor immediately."
    elif 1 <= prediction < 2:
        return "\n!!! It seems like a mild case. !!!\nConsider home remedies.\n- Get plenty of rest and avoid strenuous activity.\n- Maintain good hygiene by washing your hands regularly."
    else:
        return "\n!!! Normal condition detected. !!!\nNo action needed. If symptoms persist or worsen, please consult a doctor."

def save_answer_to_firestore(answer):
    receiver = generate_random_sender()
    doc_ref = db.collection('botmessages').document()  
    timestamp = int(time.time() * 1000)
    doc_ref.set({
        'message': answer,
        'sender': receiver,
        'timestamp': timestamp
    })
    print(f"Saved answer to Firebase: {answer}, Sender: {receiver}")
def get_latest_user_question():
    user_messages_ref = db.collection('usermessages').order_by('timestamp', direction=firestore.Query.DESCENDING).limit(1)
    user_message = user_messages_ref.stream()

    for msg in user_message:
        return msg.to_dict().get('message', '')

    return None
def main():
    file_path = 'aidata.json'
    data = json.load(open(file_path, 'r'))

    print("Welcome to the Medical Q&A bot!")

    model = joblib.load('symptom_predictor_model.pkl')
    scaler = joblib.load('scaler.pkl')

    while True:
        user_question = get_latest_user_question()

        if user_question.lower():
            print(f"User's question: {user_question}")

            normalized_question = re.sub(r'[^\w\s]', '', user_question.lower().strip())
        
            if 'fever' in normalized_question:
                print("You mentioned fever. Asking symptom questions via Firebase...")
                user_data = collect_symptom_data_via_firestore()
                if user_data:
                    answer = predict_new_case(model, scaler, user_data)
                    print(answer)
                    save_answer_to_firestore(answer)
                else:
                    print("Failed to collect symptom data.")
            else:
                print("Sorry, I couldn't understand that question. Please try again.")
                save_answer_to_firestore("Sorry, I couldn't understand that question.")
                break
        else:
            print("No new questions found. Waiting for new questions...")

        time.sleep(5)
if __name__ == "__main__":
    main()

In [3]:
import time
import json
import re
import random
import string
import numpy as np
import firebase_admin
from firebase_admin import credentials, firestore
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import joblib

if not firebase_admin._apps:
    cred = credentials.Certificate('medi-ai-bot-firebase-adminsdk-tez3h-83a58427d1.json')
    firebase_admin.initialize_app(cred)
else:
    print("Firebase app already initialized.")

db = firestore.client()

def generate_random_sender():
    return ''.join(random.choices(string.ascii_letters + string.digits, k=10))

def send_question_to_firestore(question):
    sender = generate_random_sender()
    doc_ref = db.collection('botmessages').document()
    timestamp = int(time.time() * 1000)
    doc_ref.set({
        'message': question,
        'sender': sender,
        'timestamp': timestamp
    })
    print(f"Sent question to Firebase: {question}, Sender: {sender}")
    return timestamp

def wait_for_answer_in_firestore(sent_timestamp, timeout=10):
    start_time = time.time()
    print(f"Waiting for an answer with a timestamp greater than {sent_timestamp}")

    while time.time() - start_time < timeout:
        try:
            answers_ref = db.collection('usermessages')\
                .where('timestamp', '>', sent_timestamp)\
                .order_by('timestamp', direction=firestore.Query.ASCENDING)\
                .limit(1)
            answers = answers_ref.stream()
            for answer in answers:
                answer_data = answer.to_dict()
                print(f"Answer received: {answer_data.get('message')}")
                return answer_data.get('message', None)
        except Exception as e:
            print(f"Firestore query failed: {e}. Retrying...")
        time.sleep(2)

    print("No new answers received within the timeout period.")
    return None

def collect_symptom_data_via_firestore():
    questions = [
        "How many days have you had a fever (0 if no fever)?",
        "What is your temperature in Fahrenheit (e.g., 98.6)?",
        "Do you have any previous health issues (1 = Yes, 0 = No)?",
        "Are you experiencing vomiting (1 = Yes, 0 = No)?",
        "Do you have a cold (1 = Yes, 0 = No)?",
        "Are you feeling fatigued or unusually tired (1 = Yes, 0 = No)?",
        "Do you have a cough (1 = Yes, 0 = No)?",
        "Do you have body pain (1 = Yes, 0 = No)?"
    ]

    user_data = []

    for question in questions:
        sent_timestamp = send_question_to_firestore(question)
        answer = None

        while answer is None:
            answer = wait_for_answer_in_firestore(sent_timestamp)
            if answer is not None:
                try:
                    if "temperature" in question:
                        user_data.append(float(answer))
                    else:
                        user_data.append(int(answer))
                    print(f"Received answer for '{question}': {answer}")
                except ValueError:
                    print(f"Invalid response for '{question}'. Retrying...")
                    answer = None
            else:
                print(f"No response for '{question}' yet. Retrying...")

    return user_data

def predict_new_case(model, scaler, user_data):
    user_data_scaled = scaler.transform(np.array(user_data).reshape(1, -1))
    prediction = model.predict(user_data_scaled)
    
    if prediction >= 2:
        return "\n!!! Serious condition detected! !!!\nIt is advisable to consult a doctor immediately."
    elif 1 <= prediction < 2:
        return "\n!!! It seems like a mild case. !!!\nConsider home remedies.\n- Get plenty of rest and avoid strenuous activity.\n- Maintain good hygiene by washing your hands regularly."
    else:
        return "\n!!! Normal condition detected. !!!\nNo action needed. If symptoms persist or worsen, please consult a doctor."

def save_answer_to_firestore(answer):
    receiver = generate_random_sender()
    doc_ref = db.collection('botmessages').document()  
    timestamp = int(time.time() * 1000)
    doc_ref.set({
        'message': answer,
        'sender': receiver,
        'timestamp': timestamp
    })
    print(f"Saved answer to Firebase: {answer}, Sender: {receiver}")

def get_latest_user_question():
    user_messages_ref = db.collection('usermessages').order_by('timestamp', direction=firestore.Query.DESCENDING).limit(1)
    user_message = user_messages_ref.stream()

    for msg in user_message:
        return msg.to_dict().get('message', '')

    return None

def find_best_matching_question(user_question, dataset_questions):
    vectorizer = TfidfVectorizer(stop_words='english')
    all_questions = [user_question] + dataset_questions
    tfidf_matrix = vectorizer.fit_transform(all_questions)
    cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()
    best_match_index = np.argmax(cosine_similarities)
    return best_match_index, cosine_similarities[best_match_index]

def load_dataset_from_json(file_path):
    with open(file_path, 'r') as f:
        dataset = json.load(f)
    dataset_questions = [entry['prompt'] for entry in dataset]
    dataset_answers = {entry['prompt']: entry['completion'] for entry in dataset}
    return dataset_questions, dataset_answers

def main():
    print("Welcome to the Medical Q&A bot!")

    model = joblib.load('symptom_predictor_model.pkl')
    scaler = joblib.load('scaler.pkl')

    while True:
        user_question = get_latest_user_question()

        if user_question:
            print(f"User's question: {user_question}")
            normalized_question = re.sub(r'[^\w\s]', '', user_question.lower().strip())

            if 'fever' in normalized_question:
                print("You mentioned fever. Asking symptom questions via Firebase...")
                user_data = collect_symptom_data_via_firestore()
                if user_data:
                    answer = predict_new_case(model, scaler, user_data)
                    print(answer)
                    save_answer_to_firestore(answer)
                else:
                    print("Failed to collect symptom data.")
            elif 'exit' in normalized_question:
                answer = "Thank You!!!"
                save_answer_to_firestore(answer)
                break
            else:
                dataset_questions, dataset_answers = load_dataset_from_json('aidata.json')
                best_match_index, similarity = find_best_matching_question(normalized_question, dataset_questions)

                if similarity > 0.5:
                    matched_question = dataset_questions[best_match_index]
                    answer = dataset_answers[matched_question]
                    print(f"Best matching question: {matched_question} (Similarity: {similarity})")
                    print(f"Answer: {answer}")
                    save_answer_to_firestore(answer)
                    break
                else:
                    print("Sorry, I couldn't find a match for that question. Please try again.")
                    save_answer_to_firestore("Sorry, I couldn't understand that question.")

        else:
            print("No new questions found. Asking if I can assist further...")
            # follow_up_question = "Let me know if I can assist you further!"
            # send_question_to_firestore(follow_up_question)

            answer = wait_for_answer_in_firestore(sent_timestamp=None, timeout=10)

            if answer is None:
                print("No new question. Continuing to wait for new questions...")
            else:
                print(f"User responded: {answer}")

        time.sleep(5)

if __name__ == "__main__":
    main()


Firebase app already initialized.
Welcome to the Medical Q&A bot!
User's question: hello
Sorry, I couldn't find a match for that question. Please try again.
Saved answer to Firebase: Sorry, I couldn't understand that question., Sender: 9N4OBcP25k


KeyboardInterrupt: 